In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from ST import *
from SC import *
from P0 import compileString
from HM import LinkedList, Node
from CGwat import genShowHeap

Importing Jupyter notebook from ST.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from HM.ipynb
Importing Jupyter notebook from CGwat.ipynb


In [2]:
def runpywasm(wasmfile):
    import pywasm
    def write(i): print(i)
    def writeln(): print('\n')
    def read(): return int(input())
    

    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

`Demo 1: first fit`

In [3]:
compileString("""
program p;
  type a = ^integer;
  type b = ^array [1..5] of integer;
  type c = ^record t : integer end; 
  var x : a; 
  var y : b; 
  var z : c; 
  var n : c; 
  var m : b; 
  var i : integer;
  begin
      i := 2;
      while i > 0 do
         new(x);
         i := i - 1
      end
      new(x); {point to address 20}
      new(y); {point to address 24}
      new(z); {point to address 44}
      dispose(y); {address 24 to 40 freed}
      new(n); {point to address 24}
      new(m)  {point to address 48}
      
  end
""", dstfn = "test.wat")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $i (mut i32) i32.const 0)
(func $program
i32.const 2
global.set $i
loop
global.get $i
i32.const 0
i32.gt_s
if
i32.const 0
i32.const 20
i32.store
br 1
end
end
global.get $i
i32.const 1
i32.sub
global.set $i
)
(memory 1)
(start $program)
)


In [4]:
!wat2wasm test.wat

In [ ]:
runpywasm("test.wasm")

In [ ]:
genShowHeap()

`Demo 2: assign value to heap variable and retrieve value from heap variable`

In [7]:
compileString("""
program p;
  type a = ^integer;
  type b = ^array [1..5] of integer;
  type i = integer;
  var x : a;
  var y : b;
  var n : i;
  begin
      new(x);
      x^ := 10;
      write(x^); {should print out 10}
      new(y);
      y[4]^ := 15;
      n := y[4]^;
      write(n) {should print out 15}
  end
""", dstfn = "test.wat")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $n (mut i32) i32.const 0)
(func $program
i32.const 0
i32.const 8
i32.store
i32.const 0
i32.load
i32.const 10
i32.store
i32.const 0
i32.load
i32.load
call $write
i32.const 4
i32.const 12
i32.store
i32.const 4
i32.load
i32.const 60
i32.add
i32.const 15
i32.store
i32.const 4
i32.load
i32.const 60
i32.add
i32.load
global.set $n
global.get $n
call $write
)
(memory 1)
(start $program)
)


In [8]:
!wat2wasm test.wat

In [9]:
runpywasm("test.wasm")

10
15


In [10]:
genShowHeap()

adr is 8, size is 4, is_alloc: True, nxt: 
adr is 12, size is 20, is_alloc: True, nxt: 
None


`Demo 3: test case for testing unary and binary operators for heap variable`

In [10]:
compileString("""
program p;
  type a = ^array [1..5] of boolean;
  type b = boolean;
  type c = ^integer;
  type i = integer;
  var x : a;
  var y : b;
  var z : c;
  var n : i;
  begin
      new(x);
      new(z);
      x[1]^ := false;
      y := not x[1]^ and true;
      write(y); {print 1}
      z^ := 2;
      n := 10 div z^;
      write(n); {print 5}
      n := z^ * z^;
      write(n); {print 4}
      n := 10 mod z^;
      write(n) {print 0}
  end
""", dstfn = "test.wat")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $y (mut i32) i32.const 0)
(global $n (mut i32) i32.const 0)
(func $program
i32.const 0
i32.const 8
i32.store
i32.const 4
i32.const 28
i32.store
i32.const 0
i32.load
i32.const 0
i32.store
i32.const 0
i32.load
i32.load
i32.eqz
if (result i32)
i32.const 1
else
i32.const 0
end
global.set $y
global.get $y
call $write
i32.const 4
i32.load
i32.const 2
i32.store
i32.const 10
i32.const 4
i32.load
i32.load
i32.div_s
global.set $n
global.get $n
call $write
i32.const 4
i32.load
i32.load
i32.const 4
i32.load
i32.load
i32.mul
global.set $n
global.get $n
call $write
i32.const 10
i32.const 4
i32.load
i32.load
i32.rem_s
global.set $n
global.get $n
call $write
)
(memory 1)
(start $program)
)


In [11]:
!wat2wasm test.wat

In [12]:
runpywasm("test.wasm")

1
5
4
0


In [14]:
genShowHeap()

adr is 8, size is 20, is_alloc: True, nxt: 
adr is 28, size is 4, is_alloc: True, nxt: 
None
